In [27]:
import pandas as pd
# df = pd.read_csv('data/examples/full_traj_59-Scan-50%_edge_node_df.csv')  # Update 'your_data.csv' with your CSV file path
df = pd.read_parquet('data/examples/subset_10_percent_jakarta.parquet')  # Update 'your_data.csv' with your CSV file path

df.head()

,trj_id,driving_mode,osname,pingtimestamp,rawlat,rawlng,speed,bearing,accuracy
608,26626,motorcycle,android,1555888651,-6.406107,106.897543,9.17,296,3.900
1435,26626,motorcycle,android,1555888459,-6.411458,106.904691,11.53,323,6.000
2502,26626,motorcycle,android,1555888756,-6.406727,106.890785,8.19,253,3.900
3473,26626,motorcycle,android,1555888901,-6.402572,106.883660,3.25,245,3.631
3546,26626,motorcycle,android,1555888385,-6.416653,106.906933,12.84,15,4.000


In [28]:
df.rename(columns = {'trj_id':'Vehicle ID', 'pingtimestamp': 'Position Date Time', 'rawlat': 'lat', 'rawlng': 'long'}, inplace = True)

In [29]:
df.sort_values(by=["Vehicle ID", 'Position Date Time'], inplace=True)
df.reset_index(drop=True, inplace=True)

In [30]:
df.sort_values(by=["Vehicle ID", 'Position Date Time'], inplace=True)
df.reset_index(drop=True, inplace=True)
vechile_ids = df["Vehicle ID"].unique()
one_id = vechile_ids[0]
sub_df = df[df["Vehicle ID"] == one_id]
sub_df.head()

,Vehicle ID,driving_mode,osname,Position Date Time,lat,long,speed,bearing,accuracy
0,10012,car,ios,1555730862,-6.385544,106.895873,17.209999,173,5.0
1,10012,car,ios,1555730863,-6.385695,106.895859,17.180000,172,5.0
2,10012,car,ios,1555730864,-6.385853,106.895847,17.379999,171,5.0
3,10012,car,ios,1555730865,-6.386006,106.895841,16.969999,170,5.0
4,10012,car,ios,1555730868,-6.386447,106.895808,16.200001,170,5.0


In [31]:
vechile_ids = df["Vehicle ID"].unique()
for id in vechile_ids:
    one_id = vechile_ids[0]
    sub_df = df[df["Vehicle ID"] == id]
    sub_df.to_csv('data/examples/traj_'+ str(id) + '.csv')

In [11]:
from mappymatch import package_root
from mappymatch.constructs.trace import Trace

trace = Trace.from_dataframe(dataframe=df, xy=True, lat_column='lat', lon_column='long')
trace

Mappymatch Trace object
coords: [Coordinate(coordinate_id=0, x=-13630700.174495572, y=4549104.36695885, crs=('EPSG', '3857')), Coordinate(coordinate_id=1, x=-13630627.148909612, y=4548827.027959942, crs=('EPSG', '3857')), Coordinate(coordinate_id=2, x=-13630736.687288553, y=4548736.320413939, crs=('EPSG', '3857')), Coordinate(coordinate_id=3, x=-13630794.462104272, y=4548468.146204684, crs=('EPSG', '3857')), Coordinate(coordinate_id=4, x=-13630691.714214271, y=4547815.345530677, crs=('EPSG', '3857')), Coordinate(coordinate_id=5, x=-13631238.626872538, y=4547717.041661147, crs=('EPSG', '3857')), Coordinate(coordinate_id=6, x=-13631363.416021718, y=4547697.1838288335, crs=('EPSG', '3857')), Coordinate(coordinate_id=7, x=-13631560.451520422, y=4547664.932536488, crs=('EPSG', '3857')), Coordinate(coordinate_id=8, x=-13631970.663843997, y=4547598.881074882, crs=('EPSG', '3857')), Coordinate(coordinate_id=9, x=-13632250.187085377, y=4547559.588399901, crs=('EPSG', '3857')), Coordinate(coordi

In [13]:
from mappymatch.utils.plot import plot_trace

plot_trace(trace, point_color="black", line_color="yellow")

In [12]:
from mappymatch.constructs.geofence import Geofence

geofence = Geofence.from_trace(trace, padding=1e3)
geofence

In [14]:

from mappymatch.utils.plot import plot_geofence

plot_trace(trace, point_color="black", m=plot_geofence(geofence))

In [15]:
from mappymatch.maps.nx.nx_map import NxMap, NetworkType

nx_map = NxMap.from_geofence(geofence, network_type=NetworkType.DRIVE)

In [16]:
from mappymatch.utils.plot import plot_map

plot_map(nx_map)

In [17]:
from mappymatch.matchers.lcss.lcss import LCSSMatcher

matcher = LCSSMatcher(nx_map)

match_result = matcher.match_trace(trace)

In [18]:
from mappymatch.utils.plot import plot_matches

plot_matches(match_result.matches)

In [9]:
import pandas as pd
from ipyleaflet import Marker, Polyline, Circle, Map, basemaps, FullScreenControl, basemap_to_tiles, DrawControl, Polyline, Popup
from shapely.geometry import LineString
from mappymatch.utils.crs import LATLON_CRS, XY_CRS
from shapely.geometry import Point

import geopandas as gpd
traj_filepath = 'data/examples/sample_full_traj_59-Scan-50%_edge_node_df.csv'
print("Traj Filename: ", traj_filepath)
df = pd.read_csv(traj_filepath)  # Update 'your_data.csv' with your CSV file path
# print("Selecting one Trajectory!")
vechile_ids = df["Vehicle ID"].unique()
# one_id = vechile_ids[0]
if len(vechile_ids) > 1:
    raise Exception("Value Error: More than one trajectory is present!")
    one_id = random.choice(vechile_ids)
    sub_df = df[df["Vehicle ID"] == one_id]
else:
    one_id = vechile_ids[0]
    sub_df = df
sub_df.sort_values(by=['Position Date Time'], inplace=True)    

geometry = gpd.points_from_xy(df.long, df.lat)

# Convert to GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry=geometry)

# Set the coordinate reference system (CRS)
gdf.crs = LATLON_CRS #'EPSG:4326'  # WGS84 coordinate system

gdf = gdf.to_crs(LATLON_CRS)

Traj Filename:  data/examples/sample_full_traj_59-Scan-50%_edge_node_df.csv


In [15]:
import panel as pn
import numpy as np
import pandas as pd

# Create some sample data
data = {'x': np.arange(10), 'y': np.random.rand(10)}

# Create a DataFrame
df = pd.DataFrame(data)

# Create a column layout
column = pn.Column(df)

# Create a button widget to toggle the layout
button = pn.widgets.Button(name='Toggle Layout')

# Define a callback function to toggle the layout
def toggle_layout(event):
    if column.layout == pn.Row:
        column.layout = pn.Column
    else:
        column.layout = pn.Row

# Bind the button to the callback function
button.on_click(toggle_layout)

# Display the layout
pn.extension()
layout = pn.Column(button, column)
layout.servable()

BokehModel(combine_events=True, render_bundle={'docs_json': {'a8e5f0b6-d9f3-4bf5-bcea-d37ff2617dc3': {'version…